In [1]:
import pandas as pd
import regex as re
import requests
from bs4 import BeautifulSoup
import numpy as np

package_names, prices, prices_before_discount, days, nights, cities, adventures, natures, hill_station, water_activities, religious_list, starhotel_2, starhotel_3, starhotel_4, starhotel_5, ratings, reviews, urls = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

# Define your custom URL list
url_list = [
    'https://traveltriangle.com/family-packages/india/page/21',
    'https://traveltriangle.com/family-packages/india/page/22',
    'https://traveltriangle.com/family-packages/india/page/23',
    'https://traveltriangle.com/family-packages/india/page/24',
    'https://traveltriangle.com/family-packages/india/page/25']

for url in url_list:
    try:
        page = requests.get(url, allow_redirects=False)
        soup = BeautifulSoup(page.text, 'html.parser')


        package_class = soup.find_all('span', class_="fw9 m0 f16 pfc3")
        package_names.extend([package.text for package in package_class])

        price_class = soup.find_all('h4', class_="sfc3 m0 f24 fw9 priceVal at_newprice")

        if price_class:
            prices.extend([price.text.split()[1].split('/')[0] if len(price.text.split()) > 1 else np.nan for price in price_class])
            prices_before_discount.extend([price.text.split()[2].split('/')[0] if len(price.text.split()) > 2 else np.nan for price in price_class])
        else:
            # If price_class is empty, fill prices and prices_before_discount with NaN
            prices.extend([np.nan])
            prices_before_discount.extend([np.nan])

        duration = soup.find_all('span', class_="iblock sfc6")
        d = [duration.text for duration in duration]
        days.extend([package.split()[0] for package in d])
        nights.extend([package.split()[3] for package in d])

        city_class = soup.find_all('ul', class_="clearfix package-cities-list at_packagecity_list")
        cities.extend([city.text.split('(')[0] if city else np.nan for city in city_class])

        activities_class = soup.find_all('div', class_="col-md-6 pl8 pr0 clearfix package-tag-box")
        activities = [activity.text for activity in activities_class]

        # Loop for Adventure
        for item in activities:
            if 'Adventure' in item:
                adventures.append('Adventure')
            else:
                adventures.append(np.nan)

        # Loop for Nature
        for item in activities:
            if 'Nature' in item:
                natures.append('Nature')
            else:
                natures.append(np.nan)

        # Loop for Hill Stations
        for item in activities:
            if 'Hill Stations' in item:
                hill_station.append('Hill Stations')
            else:
                hill_station.append(np.nan)

        # Loop for Water Activities
        for item in activities:
            if 'Water Activities' in item:
                water_activities.append('Water Activities')
            else:
                water_activities.append(np.nan)

        # Loop for Religious
        for item in activities:
            if 'Religious' in item:
                religious_list.append('Religious')
            else:
                religious_list.append(np.nan)

        starhotel_class = soup.find_all('div', class_="flex alignCenter")
        stars = [star.text for star in starhotel_class]

        for item in stars:
            b = re.search(r'2 Star', item)
            if b is not None:
                starhotel_2.append(b.group(0))
            else:
                starhotel_2.append(np.nan)

        for item in stars:
            b = re.search(r'3 Star', item)
            if b is not None:
                starhotel_3.append(b.group(0))
            else:
                starhotel_3.append(np.nan)

        for item in stars:
            b = re.search(r'4 Star', item)
            if b is not None:
                starhotel_4.append(b.group(0))
            else:
                starhotel_4.append(np.nan)

        for item in stars:
            b = re.search(r'5 Star', item)
            if b is not None:
                starhotel_5.append(b.group(0))
            else:
                starhotel_5.append(np.nan)

        link_class = soup.find_all('a', class_="block fw7 p8 pl15 pr15 text-center link-pri at_packageviewdetailbtn")

        # Find all matches of the specified pattern
        matches = [re.findall(r'href="([^"]*)"', str(tag))[0] for tag in link_class]
        urls.extend(["https://traveltriangle.com" + match for match in matches])

        # Loop for Reviews and Ratings
        reviews, ratings = [], []

        for i in urls:
            try:
                page = requests.get(i, allow_redirects=False)
                soup = BeautifulSoup(page.text, 'html.parser')

                # Find the <p> tag with class "f12 at_package_count"
                p_tag = soup.find('p', class_="f12 at_package_count")

                # Check if the <p> tag is found
                if p_tag:
                    # Find all <span> elements within the <p> tag
                    spans = p_tag.find_all('span')

                    # Check if there are at least three <span> elements
                    if len(spans) >= 3:
                        reviews.append(spans[4].text if spans[4] else np.nan)
                        ratings.append(spans[3].text if spans[3] else np.nan)
                    else:
                        # Fill with NaN
                        reviews.append(np.nan)
                        ratings.append(np.nan)
                else:
                    # Fill with NaN
                    reviews.append(np.nan)
                    ratings.append(np.nan)

            except requests.RequestException as e:
                print(f"Error fetching URL: {i}, {e}")
                # Fill with NaN
                reviews.append(np.nan)
                ratings.append(np.nan)

       

        print(f"Scraping completed for URL: {url}")

    except requests.RequestException as e:
        print(f"Error fetching URL: {url}, {e}")


Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/21
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/22
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/23
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/24


C:\Users\mdimr\AppData\Local\Temp\ipykernel_1212\815765219.py:125: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(page.text, 'html.parser')


Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/25


In [2]:
d={'Package_Names':package_names,
   'Prices_Before_Discount': prices_before_discount, 
   'Prices_After_Discount':prices,
   'Days':days,
   'Nights':nights, 
   'Cities':cities,
   'Adventures':adventures, 
   'Nature':natures, 
   'Hill_Station':hill_station, 
   'Water_Activities':water_activities,
  'Religious':religious_list, 
   'StarHotel_2':starhotel_2, 
   'StarHotel_3':starhotel_3, 
   'StarHotel_4':starhotel_4, 
   'StarHotel_5':starhotel_5, 
   'Ratings':ratings,
   'Reviews':reviews}

In [3]:
df21_25=pd.DataFrame(d)

In [4]:
df21_25.to_csv(r"C:\Users\mdimr\OneDrive\Desktop\webscrapping\df21_25.csv")

In [6]:
df21_25.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Package_Names           125 non-null    object 
 1   Prices_Before_Discount  124 non-null    object 
 2   Prices_After_Discount   125 non-null    object 
 3   Days                    125 non-null    object 
 4   Nights                  125 non-null    object 
 5   Cities                  125 non-null    object 
 6   Adventures              90 non-null     object 
 7   Nature                  111 non-null    object 
 8   Hill_Station            0 non-null      float64
 9   Water_Activities        18 non-null     object 
 10  Religious               48 non-null     object 
 11  StarHotel_2             15 non-null     object 
 12  StarHotel_3             97 non-null     object 
 13  StarHotel_4             17 non-null     object 
 14  StarHotel_5             4 non-null      ob

In [8]:
import pandas as pd
import regex as re
import requests
from bs4 import BeautifulSoup
import numpy as np

package_names, prices, prices_before_discount, days, nights, cities, adventures, natures, hill_station, water_activities, religious_list, starhotel_2, starhotel_3, starhotel_4, starhotel_5, ratings, reviews, urls = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

# Define your custom URL list
url_list = [
    'https://traveltriangle.com/family-packages/india/page/26',
    'https://traveltriangle.com/family-packages/india/page/27',
    'https://traveltriangle.com/family-packages/india/page/28']
    #'https://traveltriangle.com/family-packages/india/page/24',
    #'https://traveltriangle.com/family-packages/india/page/25'


for url in url_list:
    try:
        page = requests.get(url, allow_redirects=False)
        soup = BeautifulSoup(page.text, 'html.parser')


        package_class = soup.find_all('span', class_="fw9 m0 f16 pfc3")
        package_names.extend([package.text for package in package_class])

        price_class = soup.find_all('h4', class_="sfc3 m0 f24 fw9 priceVal at_newprice")

        if price_class:
            prices.extend([price.text.split()[1].split('/')[0] if len(price.text.split()) > 1 else np.nan for price in price_class])
            prices_before_discount.extend([price.text.split()[2].split('/')[0] if len(price.text.split()) > 2 else np.nan for price in price_class])
        else:
            # If price_class is empty, fill prices and prices_before_discount with NaN
            prices.extend([np.nan])
            prices_before_discount.extend([np.nan])

        duration = soup.find_all('span', class_="iblock sfc6")
        d = [duration.text for duration in duration]
        days.extend([package.split()[0] for package in d])
        nights.extend([package.split()[3] for package in d])

        city_class = soup.find_all('ul', class_="clearfix package-cities-list at_packagecity_list")
        cities.extend([city.text.split('(')[0] if city else np.nan for city in city_class])

        activities_class = soup.find_all('div', class_="col-md-6 pl8 pr0 clearfix package-tag-box")
        activities = [activity.text for activity in activities_class]

        # Loop for Adventure
        for item in activities:
            if 'Adventure' in item:
                adventures.append('Adventure')
            else:
                adventures.append(np.nan)

        # Loop for Nature
        for item in activities:
            if 'Nature' in item:
                natures.append('Nature')
            else:
                natures.append(np.nan)

        # Loop for Hill Stations
        for item in activities:
            if 'Hill Stations' in item:
                hill_station.append('Hill Stations')
            else:
                hill_station.append(np.nan)

        # Loop for Water Activities
        for item in activities:
            if 'Water Activities' in item:
                water_activities.append('Water Activities')
            else:
                water_activities.append(np.nan)

        # Loop for Religious
        for item in activities:
            if 'Religious' in item:
                religious_list.append('Religious')
            else:
                religious_list.append(np.nan)

        starhotel_class = soup.find_all('div', class_="flex alignCenter")
        stars = [star.text for star in starhotel_class]

        for item in stars:
            b = re.search(r'2 Star', item)
            if b is not None:
                starhotel_2.append(b.group(0))
            else:
                starhotel_2.append(np.nan)

        for item in stars:
            b = re.search(r'3 Star', item)
            if b is not None:
                starhotel_3.append(b.group(0))
            else:
                starhotel_3.append(np.nan)

        for item in stars:
            b = re.search(r'4 Star', item)
            if b is not None:
                starhotel_4.append(b.group(0))
            else:
                starhotel_4.append(np.nan)

        for item in stars:
            b = re.search(r'5 Star', item)
            if b is not None:
                starhotel_5.append(b.group(0))
            else:
                starhotel_5.append(np.nan)

        link_class = soup.find_all('a', class_="block fw7 p8 pl15 pr15 text-center link-pri at_packageviewdetailbtn")

        # Find all matches of the specified pattern
        matches = [re.findall(r'href="([^"]*)"', str(tag))[0] for tag in link_class]
        urls.extend(["https://traveltriangle.com" + match for match in matches])

        # Loop for Reviews and Ratings
        reviews, ratings = [], []

        for i in urls:
            try:
                page = requests.get(i, allow_redirects=False)
                soup = BeautifulSoup(page.text, 'html.parser')

                # Find the <p> tag with class "f12 at_package_count"
                p_tag = soup.find('p', class_="f12 at_package_count")

                # Check if the <p> tag is found
                if p_tag:
                    # Find all <span> elements within the <p> tag
                    spans = p_tag.find_all('span')

                    # Check if there are at least three <span> elements
                    if len(spans) >= 3:
                        reviews.append(spans[4].text if spans[4] else np.nan)
                        ratings.append(spans[3].text if spans[3] else np.nan)
                    else:
                        # Fill with NaN
                        reviews.append(np.nan)
                        ratings.append(np.nan)
                else:
                    # Fill with NaN
                    reviews.append(np.nan)
                    ratings.append(np.nan)

            except requests.RequestException as e:
                print(f"Error fetching URL: {i}, {e}")
                # Fill with NaN
                reviews.append(np.nan)
                ratings.append(np.nan)

       

        print(f"Scraping completed for URL: {url}")

    except requests.RequestException as e:
        print(f"Error fetching URL: {url}, {e}")


C:\Users\mdimr\AppData\Local\Temp\ipykernel_1212\1602551568.py:126: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(page.text, 'html.parser')


Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/26
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/27
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/28


In [9]:
d={'Package_Names':package_names,
   'Prices_Before_Discount': prices_before_discount, 
   'Prices_After_Discount':prices,
   'Days':days,
   'Nights':nights, 
   'Cities':cities,
   'Adventures':adventures, 
   'Nature':natures, 
   'Hill_Station':hill_station, 
   'Water_Activities':water_activities,
  'Religious':religious_list, 
   'StarHotel_2':starhotel_2, 
   'StarHotel_3':starhotel_3, 
   'StarHotel_4':starhotel_4, 
   'StarHotel_5':starhotel_5, 
   'Ratings':ratings,
   'Reviews':reviews}

In [10]:
df26_28=pd.DataFrame(d)

In [11]:
df26_28.to_csv(r"C:\Users\mdimr\OneDrive\Desktop\webscrapping\df26_28.csv")

In [12]:
df26_28

,Package_Names,Prices_Before_Discount,Prices_After_Discount,Days,Nights,Cities,Adventures,Nature,Hill_Station,Water_Activities,Religious,StarHotel_2,StarHotel_3,StarHotel_4,StarHotel_5,Ratings,Reviews
0,Golden Triangle Tour Package For 5 Nights 6 Days,"34,782","31,999",6,5,New Delhi,NaN,NaN,NaN,NaN,NaN,NaN,3 Star,NaN,NaN,4.1,2931
1,Fun-Filled Mount Abu Family Tour,"20,000","18,000",4,3,Udaipur,NaN,Nature,NaN,NaN,Religious,NaN,3 Star,NaN,NaN,4.2,843
2,Exciting Himachal Tour Package By Volvo Bus,"32,609","30,000",9,8,Shimla,Adventure,Nature,NaN,NaN,NaN,NaN,3 Star,NaN,NaN,4,4042
3,Nameri National Park Tour Package,"15,760","14,499",3,2,Balipara,Adventure,Nature,NaN,NaN,NaN,NaN,3 Star,NaN,NaN,NaN,NaN
4,Best South India Religious Tour Package For A ...,"19,480","18,116",3,2,Chennai,NaN,Nature,NaN,NaN,Religious,NaN,3 Star,NaN,NaN,4.3,108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,"Beautiful Landscapes, waterfalls, temples: Nor...","20,108","18,499",6,5,Shillong,Adventure,Nature,NaN,NaN,NaN,NaN,3 Star,NaN,NaN,4.2,398
71,Travelerâs Paradise: Sikkim,"28,806","26,502",6,5,Gangtok,Adventure,Nature,NaN,NaN,NaN,NaN,NaN,4 Star,NaN,4.2,8298
72,A blend of Nature and Culture: Sikkim,"28,470","26,192",7,6,Gangtok,Adventure,Nature,NaN,NaN,NaN,NaN,3 Star,NaN,NaN,4.1,8657
73,Enjoy A Grand Tour Of Magnificent Kashmir,"28,260","25,999",7,6,Srinagar,Adventure,Nature,NaN,NaN,NaN,NaN,NaN,4 Star,NaN,4.2,2114


In [13]:
import pandas as pd
import regex as re
import requests
from bs4 import BeautifulSoup
import numpy as np

package_names, prices, prices_before_discount, days, nights, cities, adventures, natures, hill_station, water_activities, religious_list, starhotel_2, starhotel_3, starhotel_4, starhotel_5, ratings, reviews, urls = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

# Define your custom URL list
url_list = [
    'https://traveltriangle.com/family-packages/india/page/16',
    'https://traveltriangle.com/family-packages/india/page/17',
    'https://traveltriangle.com/family-packages/india/page/18',
    'https://traveltriangle.com/family-packages/india/page/19',
    'https://traveltriangle.com/family-packages/india/page/20']


for url in url_list:
    try:
        page = requests.get(url, allow_redirects=False)
        soup = BeautifulSoup(page.text, 'html.parser')


        package_class = soup.find_all('span', class_="fw9 m0 f16 pfc3")
        package_names.extend([package.text for package in package_class])

        price_class = soup.find_all('h4', class_="sfc3 m0 f24 fw9 priceVal at_newprice")

        if price_class:
            prices.extend([price.text.split()[1].split('/')[0] if len(price.text.split()) > 1 else np.nan for price in price_class])
            prices_before_discount.extend([price.text.split()[2].split('/')[0] if len(price.text.split()) > 2 else np.nan for price in price_class])
        else:
            # If price_class is empty, fill prices and prices_before_discount with NaN
            prices.extend([np.nan])
            prices_before_discount.extend([np.nan])

        duration = soup.find_all('span', class_="iblock sfc6")
        d = [duration.text for duration in duration]
        days.extend([package.split()[0] for package in d])
        nights.extend([package.split()[3] for package in d])

        city_class = soup.find_all('ul', class_="clearfix package-cities-list at_packagecity_list")
        cities.extend([city.text.split('(')[0] if city else np.nan for city in city_class])

        activities_class = soup.find_all('div', class_="col-md-6 pl8 pr0 clearfix package-tag-box")
        activities = [activity.text for activity in activities_class]

        # Loop for Adventure
        for item in activities:
            if 'Adventure' in item:
                adventures.append('Adventure')
            else:
                adventures.append(np.nan)

        # Loop for Nature
        for item in activities:
            if 'Nature' in item:
                natures.append('Nature')
            else:
                natures.append(np.nan)

        # Loop for Hill Stations
        for item in activities:
            if 'Hill Stations' in item:
                hill_station.append('Hill Stations')
            else:
                hill_station.append(np.nan)

        # Loop for Water Activities
        for item in activities:
            if 'Water Activities' in item:
                water_activities.append('Water Activities')
            else:
                water_activities.append(np.nan)

        # Loop for Religious
        for item in activities:
            if 'Religious' in item:
                religious_list.append('Religious')
            else:
                religious_list.append(np.nan)

        starhotel_class = soup.find_all('div', class_="flex alignCenter")
        stars = [star.text for star in starhotel_class]

        for item in stars:
            b = re.search(r'2 Star', item)
            if b is not None:
                starhotel_2.append(b.group(0))
            else:
                starhotel_2.append(np.nan)

        for item in stars:
            b = re.search(r'3 Star', item)
            if b is not None:
                starhotel_3.append(b.group(0))
            else:
                starhotel_3.append(np.nan)

        for item in stars:
            b = re.search(r'4 Star', item)
            if b is not None:
                starhotel_4.append(b.group(0))
            else:
                starhotel_4.append(np.nan)

        for item in stars:
            b = re.search(r'5 Star', item)
            if b is not None:
                starhotel_5.append(b.group(0))
            else:
                starhotel_5.append(np.nan)

        link_class = soup.find_all('a', class_="block fw7 p8 pl15 pr15 text-center link-pri at_packageviewdetailbtn")

        # Find all matches of the specified pattern
        matches = [re.findall(r'href="([^"]*)"', str(tag))[0] for tag in link_class]
        urls.extend(["https://traveltriangle.com" + match for match in matches])

        # Loop for Reviews and Ratings
        reviews, ratings = [], []

        for i in urls:
            try:
                page = requests.get(i, allow_redirects=False)
                soup = BeautifulSoup(page.text, 'html.parser')

                # Find the <p> tag with class "f12 at_package_count"
                p_tag = soup.find('p', class_="f12 at_package_count")

                # Check if the <p> tag is found
                if p_tag:
                    # Find all <span> elements within the <p> tag
                    spans = p_tag.find_all('span')

                    # Check if there are at least three <span> elements
                    if len(spans) >= 3:
                        reviews.append(spans[4].text if spans[4] else np.nan)
                        ratings.append(spans[3].text if spans[3] else np.nan)
                    else:
                        # Fill with NaN
                        reviews.append(np.nan)
                        ratings.append(np.nan)
                else:
                    # Fill with NaN
                    reviews.append(np.nan)
                    ratings.append(np.nan)

            except requests.RequestException as e:
                print(f"Error fetching URL: {i}, {e}")
                # Fill with NaN
                reviews.append(np.nan)
                ratings.append(np.nan)

       

        print(f"Scraping completed for URL: {url}")

    except requests.RequestException as e:
        print(f"Error fetching URL: {url}, {e}")


Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/16
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/17
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/18
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/19
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/20


In [14]:
d={'Package_Names':package_names,
   'Prices_Before_Discount': prices_before_discount, 
   'Prices_After_Discount':prices,
   'Days':days,
   'Nights':nights, 
   'Cities':cities,
   'Adventures':adventures, 
   'Nature':natures, 
   'Hill_Station':hill_station, 
   'Water_Activities':water_activities,
  'Religious':religious_list, 
   'StarHotel_2':starhotel_2, 
   'StarHotel_3':starhotel_3, 
   'StarHotel_4':starhotel_4, 
   'StarHotel_5':starhotel_5, 
   'Ratings':ratings,
   'Reviews':reviews}

In [15]:
df16_20=pd.DataFrame(d)

In [16]:
df16_20.to_csv(r"C:\Users\mdimr\OneDrive\Desktop\webscrapping\df16_20.csv")

In [17]:
df16_20

,Package_Names,Prices_Before_Discount,Prices_After_Discount,Days,Nights,Cities,Adventures,Nature,Hill_Station,Water_Activities,Religious,StarHotel_2,StarHotel_3,StarHotel_4,StarHotel_5,Ratings,Reviews
0,Riveting Himachal Tour Package,"46,703","42,500",11,10,Shimla,Adventure,Nature,NaN,NaN,Religious,NaN,3 Star,NaN,NaN,4,6433
1,Pleasurable Mount Abu Honeymoon Package,"8,791","8,000",3,2,Mount Abu,Adventure,Nature,NaN,NaN,NaN,NaN,NaN,4 Star,NaN,4.3,499
2,A Serene Kausani Tour Package,"9,807","8,924",3,2,Kausani,Adventure,Nature,NaN,NaN,NaN,NaN,3 Star,NaN,NaN,4,93
3,Fabulous Lansdowne Tour Package,"8,653","7,874",2,1,Lansdowne,Adventure,Nature,NaN,NaN,NaN,NaN,3 Star,NaN,NaN,4.2,28
4,Uttarakhand Family Trip Package For 2 Nights 3...,"12,666","9,500",3,2,Jim Corbett,Adventure,Nature,NaN,NaN,NaN,NaN,NaN,4 Star,NaN,4.1,635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,Exciting Kochi Sightseeing Tour Packages,"21,679","19,728",7,6,Kochi,Adventure,Nature,NaN,NaN,NaN,NaN,NaN,4 Star,NaN,4.1,5325
121,Golden Triangle Tour Package With Udaipur,"50,537","46,999",8,7,New Delhi,Adventure,Nature,NaN,NaN,Religious,NaN,3 Star,NaN,NaN,4.1,3275
122,Shimla Tour Package By Volvo Bus,"12,903","12,000",4,3,Shimla,Adventure,Nature,NaN,NaN,NaN,NaN,3 Star,NaN,NaN,4,1231
123,Honeymoon Excursion To Enticing Dharamshala,"22,960","20,894",5,4,Dharamshala,Adventure,Nature,NaN,NaN,Religious,NaN,3 Star,NaN,NaN,4,490


In [18]:
import pandas as pd
import regex as re
import requests
from bs4 import BeautifulSoup
import numpy as np

package_names, prices, prices_before_discount, days, nights, cities, adventures, natures, hill_station, water_activities, religious_list, starhotel_2, starhotel_3, starhotel_4, starhotel_5, ratings, reviews, urls,travellers = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],[], [], [], []

# Define your custom URL list
url_list = ['https://traveltriangle.com/family-packages/india/page/6',
    'https://traveltriangle.com/family-packages/india/page/7',
    'https://traveltriangle.com/family-packages/india/page/8',
    'https://traveltriangle.com/family-packages/india/page/9',
    'https://traveltriangle.com/family-packages/india/page/10']

for url in url_list:
    try:
        page = requests.get(url, allow_redirects=False)
        soup = BeautifulSoup(page.text, 'html.parser')


        package_class = soup.find_all('span', class_="fw9 m0 f16 pfc3")
        package_names.extend([package.text for package in package_class])

        price_class = soup.find_all('h4', class_="sfc3 m0 f24 fw9 priceVal at_newprice")

        if price_class:
            prices.extend([price.text.split()[1].split('/')[0] if len(price.text.split()) > 1 else np.nan for price in price_class])
            prices_before_discount.extend([price.text.split()[2].split('/')[0] if len(price.text.split()) > 2 else np.nan for price in price_class])
        else:
            # If price_class is empty, fill prices and prices_before_discount with NaN
            prices.extend([np.nan])
            prices_before_discount.extend([np.nan])

        duration = soup.find_all('span', class_="iblock sfc6")
        d = [duration.text for duration in duration]
        days.extend([package.split()[0] for package in d])
        nights.extend([package.split()[3] for package in d])

        city_class = soup.find_all('ul', class_="clearfix package-cities-list at_packagecity_list")
        cities.extend([city.text.split('(')[0] if city else np.nan for city in city_class])

        activities_class = soup.find_all('div', class_="col-md-6 pl8 pr0 clearfix package-tag-box")
        activities = [activity.text for activity in activities_class]

        # Loop for Adventure
        for item in activities:
            if 'Adventure' in item:
                adventures.append('Adventure')
            else:
                adventures.append(np.nan)

        # Loop for Nature
        for item in activities:
            if 'Nature' in item:
                natures.append('Nature')
            else:
                natures.append(np.nan)

        # Loop for Hill Stations
        for item in activities:
            if 'Hill Stations' in item:
                hill_station.append('Hill Stations')
            else:
                hill_station.append(np.nan)

        # Loop for Water Activities
        for item in activities:
            if 'Water Activities' in item:
                water_activities.append('Water Activities')
            else:
                water_activities.append(np.nan)

        # Loop for Religious
        for item in activities:
            if 'Religious' in item:
                religious_list.append('Religious')
            else:
                religious_list.append(np.nan)

        starhotel_class = soup.find_all('div', class_="flex alignCenter")
        stars = [star.text for star in starhotel_class]

        for item in stars:
            b = re.search(r'2 Star', item)
            if b is not None:
                starhotel_2.append(b.group(0))
            else:
                starhotel_2.append(np.nan)

        for item in stars:
            b = re.search(r'3 Star', item)
            if b is not None:
                starhotel_3.append(b.group(0))
            else:
                starhotel_3.append(np.nan)

        for item in stars:
            b = re.search(r'4 Star', item)
            if b is not None:
                starhotel_4.append(b.group(0))
            else:
                starhotel_4.append(np.nan)

        for item in stars:
            b = re.search(r'5 Star', item)
            if b is not None:
                starhotel_5.append(b.group(0))
            else:
                starhotel_5.append(np.nan)

        link_class = soup.find_all('a', class_="block fw7 p8 pl15 pr15 text-center link-pri at_packageviewdetailbtn")

        # Find all matches of the specified pattern
        matches = [re.findall(r'href="([^"]*)"', str(tag))[0] for tag in link_class]
        urls.extend(["https://traveltriangle.com" + match for match in matches])

        # Loop for Reviews and Ratings
        reviews, ratings = [], []

        for i in urls:
            try:
                page = requests.get(i, allow_redirects=False)
                soup = BeautifulSoup(page.text, 'html.parser')

                # Find the <p> tag with class "f12 at_package_count"
                p_tag = soup.find('p', class_="f12 at_package_count")

                # Check if the <p> tag is found
                if p_tag:
                    # Find all <span> elements within the <p> tag
                    spans = p_tag.find_all('span')

                    # Check if there are at least three <span> elements
                    if len(spans) >= 3:
                        reviews.append(spans[4].text if spans[4] else np.nan)
                        ratings.append(spans[3].text if spans[3] else np.nan)
                    else:
                        # Fill with NaN
                        reviews.append(np.nan)
                        ratings.append(np.nan)
                else:
                    # Fill with NaN
                    reviews.append(np.nan)
                    ratings.append(np.nan)
                    
                    travellers_class = soup.find_all('p', class_="pfc4 f12 text-center m8")
                    travellers.extend([''.join(re.findall(r'\d+', traveller.text)) if traveller else np.nan for traveller in travellers_class])

            except requests.RequestException as e:
                print(f"Error fetching URL: {i}, {e}")
                # Fill with NaN
                reviews.append(np.nan)
                ratings.append(np.nan)

       

        print(f"Scraping completed for URL: {url}")

    except requests.RequestException as e:
        print(f"Error fetching URL: {url}, {e}")


Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/6
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/7
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/8
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/9
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/10


In [21]:
d={'Package_Names':package_names,
   'Prices_Before_Discount': prices_before_discount, 
   'Prices_After_Discount':prices,
   'Days':days,
   'Nights':nights, 
   'Cities':cities,
   'Adventures':adventures, 
   'Nature':natures, 
   'Hill_Station':hill_station, 
   'Water_Activities':water_activities,
  'Religious':religious_list, 
   'StarHotel_2':starhotel_2, 
   'StarHotel_3':starhotel_3, 
   'StarHotel_4':starhotel_4, 
   'StarHotel_5':starhotel_5, 
   'Ratings':ratings,
   'Reviews':reviews}

In [22]:
df6_10=pd.DataFrame(d)

In [23]:
df6_10.to_csv(r"C:\Users\mdimr\OneDrive\Desktop\webscrapping\df6_10.csv")

In [25]:
a=pd.read_csv(r"C:\Users\mdimr\OneDrive\Desktop\webscrapping\df1_5.csv")
b=pd.read_csv(r"C:\Users\mdimr\OneDrive\Desktop\webscrapping\df6_10.csv")
c=pd.read_csv(r"C:\Users\mdimr\OneDrive\Desktop\webscrapping\df11_15.csv")
d=pd.read_csv(r"C:\Users\mdimr\OneDrive\Desktop\webscrapping\df16_20.csv")
e=pd.read_csv(r"C:\Users\mdimr\OneDrive\Desktop\webscrapping\df21_25.csv")
f=pd.read_csv(r"C:\Users\mdimr\OneDrive\Desktop\webscrapping\df26_28.csv")
g=pd.read_csv(r"C:\Users\mdimr\OneDrive\Desktop\webscrapping\df30_39.csv")
h=pd.read_csv(r"C:\Users\mdimr\OneDrive\Desktop\webscrapping\df40_46.csv")

In [26]:
final_df = pd.concat([a,b,c,d,e,f,g,h],ignore_index=True)

In [27]:
final_df

,Unnamed: 0,Package_Names,Prices_Before_Discount,Prices_After_Discount,Days,Nights,Cities,Adventures,Nature,Hill_Station,Water_Activities,Religious,StarHotel_2,StarHotel_3,StarHotel_4,StarHotel_5,Ratings,Reviews
0,0,Coorg Package For 2 Nights And 3 Days,"17,802","16,200",3,2,Coorg,Adventure,Nature,NaN,NaN,Religious,NaN,3 Star,NaN,NaN,4.2,447.0
1,1,Best 4N 5D Andaman Sightseeing Tour Package Fo...,"36,484","33,200",5,4,Port Blair,Adventure,Nature,NaN,Water Activities,NaN,NaN,3 Star,NaN,NaN,4.1,2606.0
2,2,Bestselling Rishikesh Mussoorie Tour Package F...,"12,545","11,549",3,2,Rishikesh,Adventure,Nature,NaN,NaN,NaN,2 Star,3 Star,4 Star,NaN,4.0,323.0
3,3,Mesmerising Manali Trip Package From Chandigarh,"10,440","9,500",3,2,Manali,Adventure,Nature,NaN,Water Activities,Religious,NaN,3 Star,NaN,NaN,4.1,2321.0
4,4,Top Family Holiday Packages in Goa for an Idea...,"15,217","14,000",5,4,Calangute,Adventure,NaN,NaN,Water Activities,NaN,NaN,3 Star,NaN,NaN,4.5,47.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
889,89,Mesmerising Rajasthan Tour Package,"18,010","16,000",3,2,Jaipur,NaN,NaN,NaN,NaN,Religious,2 Star,3 Star,4 Star,5 Star,4.1,423.0
890,90,Enrapturing Munnar-Thekkady-Kumarakom Kerala F...,"16,909","14,880",6,5,Munnar,Adventure,Nature,NaN,NaN,NaN,NaN,3 Star,4 Star,5 Star,4.1,4223.0
891,91,Jammu Kashmir Family Holiday Package,"34,711","30,352",9,8,Katra,NaN,Nature,NaN,NaN,Religious,2 Star,3 Star,4 Star,NaN,4.1,2159.0
892,92,Rajasthan Tour Package For Adventure Lovers,"14,761","13,000",5,4,Ranthambore,NaN,NaN,NaN,NaN,NaN,2 Star,3 Star,4 Star,5 Star,3.8,53.0


In [28]:
final_df.to_csv(r"C:\Users\mdimr\OneDrive\Desktop\webscrapping\Pages1_40.csv")

In [29]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 894 entries, 0 to 893
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              894 non-null    int64  
 1   Package_Names           894 non-null    object 
 2   Prices_Before_Discount  893 non-null    object 
 3   Prices_After_Discount   894 non-null    object 
 4   Days                    894 non-null    int64  
 5   Nights                  894 non-null    int64  
 6   Cities                  894 non-null    object 
 7   Adventures              664 non-null    object 
 8   Nature                  789 non-null    object 
 9   Hill_Station            0 non-null      float64
 10  Water_Activities        165 non-null    object 
 11  Religious               269 non-null    object 
 12  StarHotel_2             168 non-null    object 
 13  StarHotel_3             678 non-null    object 
 14  StarHotel_4             206 non-null    ob

In [7]:
import pandas as pd
import regex as re
import requests
from bs4 import BeautifulSoup
import numpy as np





package_names, prices, prices_before_discount, days, nights, cities, adventures, natures, hill_station, water_activities, religious_list, starhotel_2, starhotel_3, starhotel_4, starhotel_5, ratings, reviews, urls,travellers = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],[], [], [], []

# Define your custom URL list
url='https://traveltriangle.com/family-packages/india/page/2'


    
page = requests.get(url, allow_redirects=False)
soup = BeautifulSoup(page.text, 'html.parser')


package_class = soup.find_all('span', class_="fw9 m0 f16 pfc3")
package_names.extend([package.text for package in package_class])

price_class = soup.find_all('h4', class_="sfc3 m0 f24 fw9 priceVal at_newprice")

if price_class:
    prices.extend([price.text.split()[1].split('/')[0] if len(price.text.split()) > 1 else np.nan for price in price_class])
    prices_before_discount.extend([price.text.split()[2].split('/')[0] if len(price.text.split()) > 2 else np.nan for price in price_class])
else:
    # If price_class is empty, fill prices and prices_before_discount with NaN
    prices.extend([np.nan])
    prices_before_discount.extend([np.nan])

duration = soup.find_all('span', class_="iblock sfc6")
d = [duration.text for duration in duration]
days.extend([package.split()[0] for package in d])
nights.extend([package.split()[3] for package in d])

city_class = soup.find_all('ul', class_="clearfix package-cities-list at_packagecity_list")
cities.extend([city.text.split('(')[0] if city else np.nan for city in city_class])

activities_class = soup.find_all('div', class_="col-md-6 pl8 pr0 clearfix package-tag-box")
activities = [activity.text for activity in activities_class]

# Loop for Adventure
for item in activities:
    if 'Adventure' in item:
        adventures.append('Adventure')
    else:
        adventures.append(np.nan)

# Loop for Nature
for item in activities:
    if 'Nature' in item:
        natures.append('Nature')
    else:
        natures.append(np.nan)

# Loop for Hill Stations
for item in activities:
    if 'Hill station' in item:
        hill_station.append('Hill Stations')
    else:
        hill_station.append(np.nan)

# Loop for Water Activities
for item in activities:
    if 'Water Activities' in item:
        water_activities.append('Water Activities')
    else:
        water_activities.append(np.nan)

# Loop for Religious
for item in activities:
    if 'Religious' in item:
        religious_list.append('Religious')
    else:
        religious_list.append(np.nan)


link_class = soup.find_all('a', class_="block fw7 p8 pl15 pr15 text-center link-pri at_packageviewdetailbtn")

# Find all matches of the specified pattern
matches = [re.findall(r'href="([^"]*)"', str(tag))[0] for tag in link_class]
urls.extend(["https://traveltriangle.com" + match for match in matches])

# Loop for Reviews and Ratings
reviews, ratings,travellers = [], [],[]

for i in urls:
    try:
        page = requests.get(i, allow_redirects=False)
        soup = BeautifulSoup(page.text, 'html.parser')

        # Find the <p> tag with class "f12 at_package_count"
        p_tag = soup.find('p', class_="f12 at_package_count")

        # Check if the <p> tag is found
        if p_tag:
            # Find all <span> elements within the <p> tag
            spans = p_tag.find_all('span')

            # Check if there are at least three <span> elements
            if len(spans) >= 3:
                reviews.append(spans[4].text if spans[4] else np.nan)
                ratings.append(spans[3].text if spans[3] else np.nan)
            else:
                # Fill with NaN
                reviews.append(np.nan)
                ratings.append(np.nan)
        else:
            # Fill with NaN
            reviews.append(np.nan)
            ratings.append(np.nan)
            
            travellers_class = soup.find_all('p', class_="pfc4 f12 text-center m8")
            travellers.extend([''.join(re.findall(r'\d+', traveller.text)) if traveller else np.nan for traveller in travellers_class])

    except requests.RequestException as e:
        print(f"Error fetching URL: {i}, {e}")
        # Fill with NaN
        reviews.append(np.nan)
        ratings.append(np.nan)
        
        
        






In [8]:
package_names

['Ecstatic Shimla 4 Night 5 Day Tour Package',
 'An Adventurous Family Trip To Jim Corbett National Park',
 'Mesmerizing Getaway To Kochi',
 'Mesmerising Lansdowne Tour Package',
 'Gangtok Family Package For 3 Days',
 'Kerala Wayanad Weekend Holiday',
 'A Goa Tour Full of Fun n Frolic',
 'Charming Kashmir Sightseeing Tour Package',
 'Amazing Andaman Tour Package',
 'Gangtok, Lachung & Darjeeling Summer Special Tour: The North Eastern Bliss ',
 'Best 2 Nights 3 Days Kodaikanal Tour Exclusive Package',
 'Exhilarating Gangtok, Kalimpong and Darjeeling Trip in Summer',
 'Exciting Rajasthan Tour Package',
 'Check Out Our Top-rated Andaman Itinerary For 4 Days For A Soulful Vacay',
 'Fascinating Manali Tour Package From Ahmedabad',
 'Resounding Rajasthan Family Package',
 'Catch The Beauty Of The Setting Sun In Alleppey',
 'Adventure Packed Rishikesh Tour Package',
 'Best Selling Ooty Mysore Tour Packages for a Relaxing Holiday',
 'Splendid Jaisalmer Tour Package',
 'A Blissful Tour In Munna

In [9]:
reviews

['1231',
 '636',
 '331',
 '28',
 '4152',
 '52',
 '2158',
 '2115',
 '18166',
 '8995',
 '105',
 '8352',
 '51',
 '2606',
 '2321',
 '336',
 '771',
 '197',
 '677',
 '336',
 '3194',
 '2158',
 '2321',
 '145',
 '7988']

In [10]:
ratings

['4',
 '4.1',
 '4.1',
 '4.2',
 '4.2',
 '4.4',
 '4.3',
 '4.2',
 '4.3',
 '4.1',
 '4.2',
 '4.1',
 '4',
 '4.1',
 '4.1',
 '4.1',
 '4.2',
 '4',
 '4.2',
 '4.1',
 '4.2',
 '4.3',
 '4.1',
 '4.1',
 '4.3']

In [13]:
len(hill_station)

25

In [19]:
import pandas as pd
import regex as re
import requests
from bs4 import BeautifulSoup
import numpy as np

package_names, prices, prices_before_discount, days, nights, cities, adventures, natures, hill_station, water_activities, religious_list, star_hotels , ratings, reviews, urls, travellers = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

# Define your custom URL list
url = 'https://traveltriangle.com/family-packages/india/page/2'

page = requests.get(url, allow_redirects=False)
soup = BeautifulSoup(page.text, 'html.parser')

package_class = soup.find_all('span', class_="fw9 m0 f16 pfc3")

package_names.extend([package.text for package in package_class])

price_class = soup.find_all('h4', class_="sfc3 m0 f24 fw9 priceVal at_newprice")

if price_class:
    prices.extend([price.text.split()[1].split('/')[0] if len(price.text.split()) > 1 else np.nan for price in price_class])
    prices_before_discount.extend([price.text.split()[2].split('/')[0] if len(price.text.split()) > 2 else np.nan for price in price_class])
else:
    # If price_class is empty, fill prices and prices_before_discount with NaN
    prices.extend([np.nan])
    prices_before_discount.extend([np.nan])

duration = soup.find_all('span', class_="iblock sfc6")
d = [duration.text for duration in duration]
days.extend([package.split()[0] for package in d])
nights.extend([package.split()[3] for package in d])

city_class = soup.find_all('ul', class_="clearfix package-cities-list at_packagecity_list")
cities.extend([city.text.split('(')[0] if city else np.nan for city in city_class])

activities_class = soup.find_all('div', class_="col-md-6 pl8 pr0 clearfix package-tag-box")
activities = [activity.text for activity in activities_class]

# Loop for Adventure
for item in activities:
    if 'Adventure' in item:
        adventures.append('Adventure')
    else:
        adventures.append(np.nan)

# Loop for Nature
for item in activities:
    if 'Nature' in item:
        natures.append('Nature')
    else:
        natures.append(np.nan)

# Loop for Hill Stations
for item in activities:
    if 'Hill station' in item:
        hill_station.append('Hill Stations')
    else:
        hill_station.append(np.nan)

# Loop for Water Activities
for item in activities:
    if 'Water Activities' in item:
        water_activities.append('Water Activities')
    else:
        water_activities.append(np.nan)

# Loop for Religious
for item in activities:
    if 'Religious' in item:
        religious_list.append('Religious')
    else:
        religious_list.append(np.nan)
        
        
        
starhotel_class = soup.find_all('div', class_="flex alignCenter")
stars = [star.text for star in starhotel_class]


for item in stars:
    # Extract star ratings using regular expression
    ratings = re.findall(r'\d Star', item)
    
    # If there are star ratings, find the maximum and append to the star_hotels list
    if ratings:
        max_star = max(map(int, re.findall(r'\d', ' '.join(ratings))))
        star_hotels.append(f'{max_star} Star')
    else:
        star_hotels.append(item)


link_class = soup.find_all('a', class_="block fw7 p8 pl15 pr15 text-center link-pri at_packageviewdetailbtn")

# Find all matches of the specified pattern
matches = [re.findall(r'href="([^"]*)"', str(tag))[0] for tag in link_class]
urls.extend(["https://traveltriangle.com" + match for match in matches])

# Loop for Reviews and Ratings
reviews, ratings, travellers = [], [], []

for i in urls:
    try:
        page = requests.get(i, allow_redirects=False)
        soup = BeautifulSoup(page.text, 'html.parser')

        # Find the <p> tag with class "f12 at_package_count"
        p_tag = soup.find('p', class_="f12 at_package_count")

        # Check if the <p> tag is found
        if p_tag:
            # Find all <span> elements within the <p> tag
            spans = p_tag.find_all('span')

            # Check if there are at least three <span> elements
            if len(spans) >= 3:
                reviews.append(spans[4].text if spans[4] else np.nan)
                ratings.append(spans[3].text if spans[3] else np.nan)
            else:
                # Fill with NaN
                reviews.append(np.nan)
                ratings.append(np.nan)
        else:
            # Fill with NaN
            reviews.append(np.nan)
            ratings.append(np.nan)

        travellers_class = soup.find_all('p', class_="pfc4 f12 text-center m8")
        travellers.extend([''.join(re.findall(r'\d+', traveller.text)) if traveller else np.nan for traveller in travellers_class])

    except requests.RequestException as e:
        print(f"Error fetching URL: {i}, {e}")
        # Fill with NaN
        reviews.append(np.nan)
        ratings.append(np.nan)




In [20]:
travellers

['34932',
 '7589',
 '44679',
 '7589',
 '3000',
 '44679',
 '13108',
 '16836',
 '20148',
 '17652',
 '5508',
 '17652',
 '8370',
 '20148',
 '34932',
 '8370',
 '44679',
 '7589',
 '5508',
 '8370',
 '44679',
 '13108',
 '34932',
 '34932',
 '4794']

In [21]:
star_hotels

['3 Star',
 '4 Star',
 '3 Star',
 '3 Star',
 '2 Star',
 '4 Star',
 '3 Star',
 '2 Star',
 '4 Star',
 '4 Star',
 '3 Star',
 '4 Star',
 '3 Star',
 '3 Star',
 '3 Star',
 '3 Star',
 '3 Star',
 '3 Star',
 '3 Star',
 '3 Star',
 '3 Star',
 '3 Star',
 '3 Star',
 '3 Star',
 '3 Star']

In [27]:
d={'Package_Names':package_names,
   'Prices_Before_Discount': prices_before_discount, 
   'Prices_After_Discount':prices,
   'Days':days,
   'Nights':nights, 
   'Cities':cities,
   'Adventures':adventures, 
   'Nature':natures, 
   'Hill_Station':hill_station, 
   'Water_Activities':water_activities,
   'Religious':religious_list, 
   'Star_Hotels':star_hotels,
   'Travellers':travellers,
   'Ratings':ratings,
   'Reviews':reviews}

In [28]:
df2=pd.DataFrame(d)

In [29]:
df2

,Package_Names,Prices_Before_Discount,Prices_After_Discount,Days,Nights,Cities,Adventures,Nature,Hill_Station,Water_Activities,Religious,Star_Hotels,Travellers,Ratings,Reviews
0,Ecstatic Shimla 4 Night 5 Day Tour Package,"14,205","12,500",5,4,Shimla,Adventure,Nature,NaN,NaN,NaN,3 Star,34932,4,1231
1,An Adventurous Family Trip To Jim Corbett Nati...,"7,933","7,219",2,1,Jim Corbett,Adventure,Nature,NaN,NaN,NaN,4 Star,7589,4.1,636
2,Mesmerizing Getaway To Kochi,"4,593","4,180",2,1,Kochi,Adventure,Nature,NaN,NaN,NaN,3 Star,44679,4.1,331
3,Mesmerising Lansdowne Tour Package,"17,308","15,750",3,2,Lansdowne,Adventure,Nature,Hill Stations,NaN,Religious,3 Star,7589,4.2,28
4,Gangtok Family Package For 3 Days,"19,163","18,013",3,2,Gangtok,Adventure,Nature,NaN,NaN,NaN,2 Star,3000,4.2,4152
5,Kerala Wayanad Weekend Holiday,"8,253","7,840",3,2,Wayanad,Adventure,Nature,Hill Stations,NaN,NaN,4 Star,44679,4.4,52
6,A Goa Tour Full of Fun n Frolic,"12,637","11,500",4,3,Goa,Adventure,NaN,NaN,Water Activities,NaN,3 Star,13108,4.3,2158
7,Charming Kashmir Sightseeing Tour Package,"30,769","28,000",8,7,Srinagar,Adventure,Nature,Hill Stations,NaN,NaN,2 Star,16836,4.2,2115
8,Amazing Andaman Tour Package,"19,216","18,163",6,5,Port Blair,NaN,Nature,NaN,Water Activities,NaN,4 Star,20148,4.3,18166
9,"Gangtok, Lachung & Darjeeling Summer Special T...","67,710","62,399",7,6,Gangtok,Adventure,Nature,Hill Stations,NaN,NaN,4 Star,17652,4.1,8995


In [26]:
df2.columns

Index(['Package_Names', 'Prices_Before_Discount', 'Prices_After_Discount',
       'Days', 'Nights', 'Cities', 'Adventures', 'Nature', 'Hill_Station',
       'Water_Activities', 'Religious', 'Star_Hotel', 'Travellers', 'Ratings',
       'Reviews'],
      dtype='object')

In [ ]:
df2.to_csv(r)

In [ ]:
import pandas as pd
import regex as re
import requests
from bs4 import BeautifulSoup
import numpy as np

package_names, prices, prices_before_discount, days, nights, cities, adventures, natures, hill_station, water_activities, religious_list, star_hotels , ratings, reviews, urls, travellers = [], [], [], [], [], [], [], [], [], [], [],

# Define your custom URL list
url_list = ['https://traveltriangle.com/family-packages/india/page/6',
    'https://traveltriangle.com/family-packages/india/page/7',
    'https://traveltriangle.com/family-packages/india/page/8',
    'https://traveltriangle.com/family-packages/india/page/9',
    'https://traveltriangle.com/family-packages/india/page/10']

for url in url_list:
    try:
        page = requests.get(url, allow_redirects=False)
        soup = BeautifulSoup(page.text, 'html.parser')


        package_class = soup.find_all('span', class_="fw9 m0 f16 pfc3")
        package_names.extend([package.text for package in package_class])

        price_class = soup.find_all('h4', class_="sfc3 m0 f24 fw9 priceVal at_newprice")

        if price_class:
            prices.extend([price.text.split()[1].split('/')[0] if len(price.text.split()) > 1 else np.nan for price in price_class])
            prices_before_discount.extend([price.text.split()[2].split('/')[0] if len(price.text.split()) > 2 else np.nan for price in price_class])
        else:
            # If price_class is empty, fill prices and prices_before_discount with NaN
            prices.extend([np.nan])
            prices_before_discount.extend([np.nan])

        duration = soup.find_all('span', class_="iblock sfc6")
        d = [duration.text for duration in duration]
        days.extend([package.split()[0] for package in d])
        nights.extend([package.split()[3] for package in d])

        city_class = soup.find_all('ul', class_="clearfix package-cities-list at_packagecity_list")
        cities.extend([city.text.split('(')[0] if city else np.nan for city in city_class])

        activities_class = soup.find_all('div', class_="col-md-6 pl8 pr0 clearfix package-tag-box")
        activities = [activity.text for activity in activities_class]

        # Loop for Adventure
        for item in activities:
            if 'Adventure' in item:
                adventures.append('Adventure')
            else:
                adventures.append(np.nan)

        # Loop for Nature
        for item in activities:
            if 'Nature' in item:
                natures.append('Nature')
            else:
                natures.append(np.nan)

        # Loop for Hill Stations
        for item in activities:
            if 'Hill station' in item:
                hill_station.append('Hill Stations')
            else:
                hill_station.append(np.nan)

        # Loop for Water Activities
        for item in activities:
            if 'Water Activities' in item:
                water_activities.append('Water Activities')
            else:
                water_activities.append(np.nan)

        # Loop for Religious
        for item in activities:
            if 'Religious' in item:
                religious_list.append('Religious')
            else:
                religious_list.append(np.nan)

        starhotel_class = soup.find_all('div', class_="flex alignCenter")
        stars = [star.text for star in starhotel_class]

        for item in stars:
            # Extract star ratings using regular expression
            ratings = re.findall(r'\d Star', item)

            # If there are star ratings, find the maximum and append to the star_hotels list
            if ratings:
                max_star = max(map(int, re.findall(r'\d', ' '.join(ratings))))
                star_hotels.append(f'{max_star} Star')
            else:
                star_hotels.append(item)

        link_class = soup.find_all('a', class_="block fw7 p8 pl15 pr15 text-center link-pri at_packageviewdetailbtn")

        # Find all matches of the specified pattern
        matches = [re.findall(r'href="([^"]*)"', str(tag))[0] for tag in link_class]
        urls.extend(["https://traveltriangle.com" + match for match in matches])

        # Loop for Reviews and Ratings
        reviews, ratings = [], []

        for i in urls:
            try:
                page = requests.get(i, allow_redirects=False)
                soup = BeautifulSoup(page.text, 'html.parser')

                # Find the <p> tag with class "f12 at_package_count"
                p_tag = soup.find('p', class_="f12 at_package_count")

                # Check if the <p> tag is found
        if p_tag:
            # Find all <span> elements within the <p> tag
            spans = p_tag.find_all('span')

            # Check if there are at least three <span> elements
            if len(spans) >= 3:
                reviews.append(spans[4].text if spans[4] else np.nan)
                ratings.append(spans[3].text if spans[3] else np.nan)
            else:
                # Fill with NaN
                reviews.append(np.nan)
                ratings.append(np.nan)
        else:
            # Fill with NaN
            reviews.append(np.nan)
            ratings.append(np.nan)

        travellers_class = soup.find_all('p', class_="pfc4 f12 text-center m8")
        travellers.extend([''.join(re.findall(r'\d+', traveller.text)) if traveller else np.nan for traveller in travellers_class])

                   

            except requests.RequestException as e:
                print(f"Error fetching URL: {i}, {e}")
                # Fill with NaN
                reviews.append(np.nan)
                ratings.append(np.nan)

       

        print(f"Scraping completed for URL: {url}")

    except requests.RequestException as e:
        print(f"Error fetching URL: {url}, {e}")


In [46]:
import pandas as pd
import regex as re
import requests
from bs4 import BeautifulSoup
import numpy as np

package_names, prices, prices_before_discount, days, nights, cities, adventures, natures, hill_station, water_activities, religious_list, star_hotels, ratings, reviews, urls, travellers = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

# Define your custom URL list
url_list = ['https://traveltriangle.com/family-packages/india/page/6',
            'https://traveltriangle.com/family-packages/india/page/7',
            'https://traveltriangle.com/family-packages/india/page/8',
            'https://traveltriangle.com/family-packages/india/page/9',
            'https://traveltriangle.com/family-packages/india/page/10']

for url in url_list:
    try:
        page = requests.get(url, allow_redirects=False)
        soup = BeautifulSoup(page.text, 'html.parser')

        package_class = soup.find_all('span', class_="fw9 m0 f16 pfc3")
        package_names.extend([package.text for package in package_class])

        price_class = soup.find_all('h4', class_="sfc3 m0 f24 fw9 priceVal at_newprice")

        if price_class:
            prices.extend([price.text.split()[1].split('/')[0] if len(price.text.split()) > 1 else np.nan for price in price_class])
            prices_before_discount.extend([price.text.split()[2].split('/')[0] if len(price.text.split()) > 2 else np.nan for price in price_class])
        else:
            # If price_class is empty, fill prices and prices_before_discount with NaN
            prices.extend([np.nan])
            prices_before_discount.extend([np.nan])

        duration = soup.find_all('span', class_="iblock sfc6")
        d = [duration.text for duration in duration]
        days.extend([package.split()[0] for package in d])
        nights.extend([package.split()[3] for package in d])

        city_class = soup.find_all('ul', class_="clearfix package-cities-list at_packagecity_list")
        cities.extend([city.text.split('(')[0] if city else np.nan for city in city_class])

        activities_class = soup.find_all('div', class_="col-md-6 pl8 pr0 clearfix package-tag-box")
        activities = [activity.text for activity in activities_class]

        # Loop for Adventure
        for item in activities:
            if 'Adventure' in item:
                adventures.append('Adventure')
            else:
                adventures.append(np.nan)

        # Loop for Nature
        for item in activities:
            if 'Nature' in item:
                natures.append('Nature')
            else:
                natures.append(np.nan)

        # Loop for Hill Stations
        for item in activities:
            if 'Hill station' in item:
                hill_station.append('Hill Stations')
            else:
                hill_station.append(np.nan)

        # Loop for Water Activities
        for item in activities:
            if 'Water Activities' in item:
                water_activities.append('Water Activities')
            else:
                water_activities.append(np.nan)

        # Loop for Religious
        for item in activities:
            if 'Religious' in item:
                religious_list.append('Religious')
            else:
                religious_list.append(np.nan)

        starhotel_class = soup.find_all('div', class_="flex alignCenter")
        stars = [star.text for star in starhotel_class]

        for item in stars:
            # Extract star ratings using regular expression
            ratings = re.findall(r'\d Star', item)

            # If there are star ratings, find the maximum and append to the star_hotels list
            if ratings:
                max_star = max(map(int, re.findall(r'\d', ' '.join(ratings))))
                star_hotels.append(f'{max_star} Star')
            else:
                star_hotels.append(item)

        link_class = soup.find_all('a', class_="block fw7 p8 pl15 pr15 text-center link-pri at_packageviewdetailbtn")

        # Find all matches of the specified pattern
        matches = [re.findall(r'href="([^"]*)"', str(tag))[0] for tag in link_class]
        urls.extend(["https://traveltriangle.com" + match for match in matches])

        # Loop for Reviews and Ratings
        reviews, ratings,travellers = [], [],[]

        for i in urls:
            try:
                page = requests.get(i, allow_redirects=False)
                soup = BeautifulSoup(page.text, 'html.parser')

                # Find the <p> tag with class "f12 at_package_count"
                p_tag = soup.find('p', class_="f12 at_package_count")

                # Check if the <p> tag is found
                if p_tag:
                    # Find all <span> elements within the <p> tag
                    spans = p_tag.find_all('span')

                    # Check if there are at least three <span> elements
                    if len(spans) >= 3:
                        reviews.append(spans[4].text if spans[4] else np.nan)
                        ratings.append(spans[3].text if spans[3] else np.nan)
                    else:
                        # Fill with NaN
                        reviews.append(np.nan)
                        ratings.append(np.nan)
                else:
                    # Fill with NaN
                    reviews.append(np.nan)
                    ratings.append(np.nan)

                travellers_class = soup.find_all('p', class_="pfc4 f12 text-center m8")
                travellers.extend([''.join(re.findall(r'\d+', traveller.text)) if traveller else np.nan for traveller in travellers_class])

            except requests.RequestException as e:
                print(f"Error fetching URL: {i}, {e}")
                # Fill with NaN
                reviews.append(np.nan)
                ratings.append(np.nan)

        print(f"Scraping completed for URL: {url}")

    except requests.RequestException as e:
        print(f"Error fetching URL: {url}, {e}")


Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/6
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/7
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/8
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/9
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/10


In [47]:
d={'Package_Names':package_names,
   'Prices_Before_Discount': prices_before_discount, 
   'Prices_After_Discount':prices,
   'Days':days,
   'Nights':nights, 
   'Cities':cities,
   'Adventures':adventures, 
   'Nature':natures, 
   'Hill_Station':hill_station, 
   'Water_Activities':water_activities,
   'Religious':religious_list, 
   'Star_Hotels':star_hotels,
   'Travellers':travellers,
   'Ratings':ratings,
   'Reviews':reviews}

In [48]:
WebPage6_10=pd.DataFrame(d)

In [49]:
WebPage6_10.to_csv(r"C:\Users\mdimr\OneDrive\Desktop\webscrapping\WebPage6_10.csv")

### Pages 11-15

In [50]:
import pandas as pd
import regex as re
import requests
from bs4 import BeautifulSoup
import numpy as np

package_names, prices, prices_before_discount, days, nights, cities, adventures, natures, hill_station, water_activities, religious_list, star_hotels, ratings, reviews, urls, travellers = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

# Define your custom URL list
url_list = ['https://traveltriangle.com/family-packages/india/page/11',
            'https://traveltriangle.com/family-packages/india/page/12',
            'https://traveltriangle.com/family-packages/india/page/13',
            'https://traveltriangle.com/family-packages/india/page/14',
            'https://traveltriangle.com/family-packages/india/page/15']

for url in url_list:
    try:
        page = requests.get(url, allow_redirects=False)
        soup = BeautifulSoup(page.text, 'html.parser')

        package_class = soup.find_all('span', class_="fw9 m0 f16 pfc3")
        package_names.extend([package.text for package in package_class])

        price_class = soup.find_all('h4', class_="sfc3 m0 f24 fw9 priceVal at_newprice")

        if price_class:
            prices.extend([price.text.split()[1].split('/')[0] if len(price.text.split()) > 1 else np.nan for price in price_class])
            prices_before_discount.extend([price.text.split()[2].split('/')[0] if len(price.text.split()) > 2 else np.nan for price in price_class])
        else:
            # If price_class is empty, fill prices and prices_before_discount with NaN
            prices.extend([np.nan])
            prices_before_discount.extend([np.nan])

        duration = soup.find_all('span', class_="iblock sfc6")
        d = [duration.text for duration in duration]
        days.extend([package.split()[0] for package in d])
        nights.extend([package.split()[3] for package in d])

        city_class = soup.find_all('ul', class_="clearfix package-cities-list at_packagecity_list")
        cities.extend([city.text.split('(')[0] if city else np.nan for city in city_class])

        activities_class = soup.find_all('div', class_="col-md-6 pl8 pr0 clearfix package-tag-box")
        activities = [activity.text for activity in activities_class]

        # Loop for Adventure
        for item in activities:
            if 'Adventure' in item:
                adventures.append('Adventure')
            else:
                adventures.append(np.nan)

        # Loop for Nature
        for item in activities:
            if 'Nature' in item:
                natures.append('Nature')
            else:
                natures.append(np.nan)

        # Loop for Hill Stations
        for item in activities:
            if 'Hill station' in item:
                hill_station.append('Hill Stations')
            else:
                hill_station.append(np.nan)

        # Loop for Water Activities
        for item in activities:
            if 'Water Activities' in item:
                water_activities.append('Water Activities')
            else:
                water_activities.append(np.nan)

        # Loop for Religious
        for item in activities:
            if 'Religious' in item:
                religious_list.append('Religious')
            else:
                religious_list.append(np.nan)

        starhotel_class = soup.find_all('div', class_="flex alignCenter")
        stars = [star.text for star in starhotel_class]

        for item in stars:
            # Extract star ratings using regular expression
            ratings = re.findall(r'\d Star', item)

            # If there are star ratings, find the maximum and append to the star_hotels list
            if ratings:
                max_star = max(map(int, re.findall(r'\d', ' '.join(ratings))))
                star_hotels.append(f'{max_star} Star')
            else:
                star_hotels.append(item)

        link_class = soup.find_all('a', class_="block fw7 p8 pl15 pr15 text-center link-pri at_packageviewdetailbtn")

        # Find all matches of the specified pattern
        matches = [re.findall(r'href="([^"]*)"', str(tag))[0] for tag in link_class]
        urls.extend(["https://traveltriangle.com" + match for match in matches])

        # Loop for Reviews and Ratings
        reviews, ratings,travellers = [], [],[]

        for i in urls:
            try:
                page = requests.get(i, allow_redirects=False)
                soup = BeautifulSoup(page.text, 'html.parser')

                # Find the <p> tag with class "f12 at_package_count"
                p_tag = soup.find('p', class_="f12 at_package_count")

                # Check if the <p> tag is found
                if p_tag:
                    # Find all <span> elements within the <p> tag
                    spans = p_tag.find_all('span')

                    # Check if there are at least three <span> elements
                    if len(spans) >= 3:
                        reviews.append(spans[4].text if spans[4] else np.nan)
                        ratings.append(spans[3].text if spans[3] else np.nan)
                    else:
                        # Fill with NaN
                        reviews.append(np.nan)
                        ratings.append(np.nan)
                else:
                    # Fill with NaN
                    reviews.append(np.nan)
                    ratings.append(np.nan)

                travellers_class = soup.find_all('p', class_="pfc4 f12 text-center m8")
                travellers.extend([''.join(re.findall(r'\d+', traveller.text)) if traveller else np.nan for traveller in travellers_class])

            except requests.RequestException as e:
                print(f"Error fetching URL: {i}, {e}")
                # Fill with NaN
                reviews.append(np.nan)
                ratings.append(np.nan)

        print(f"Scraping completed for URL: {url}")

    except requests.RequestException as e:
        print(f"Error fetching URL: {url}, {e}")


Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/11
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/12
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/13
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/14
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/15


In [51]:
d={'Package_Names':package_names,
   'Prices_Before_Discount': prices_before_discount, 
   'Prices_After_Discount':prices,
   'Days':days,
   'Nights':nights, 
   'Cities':cities,
   'Adventures':adventures, 
   'Nature':natures, 
   'Hill_Station':hill_station, 
   'Water_Activities':water_activities,
   'Religious':religious_list, 
   'Star_Hotels':star_hotels,
   'Travellers':travellers,
   'Ratings':ratings,
   'Reviews':reviews}

In [52]:
WebPage11_15=pd.DataFrame(d)

In [53]:
WebPage11_15.to_csv(r"C:\Users\mdimr\OneDrive\Desktop\webscrapping\WebPage11_15.csv")

### Pages 30-40

In [55]:
import pandas as pd
import regex as re
import requests
from bs4 import BeautifulSoup
import numpy as np

package_names, prices, prices_before_discount, days, nights, cities, adventures, natures, hill_station, water_activities, religious_list, star_hotels, ratings, reviews, urls, travellers = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

# Define your custom URL list
url_list = [
    'https://traveltriangle.com/family-packages/india/page/35',
    'https://traveltriangle.com/family-packages/india/page/36',
    'https://traveltriangle.com/family-packages/india/page/38',
    'https://traveltriangle.com/family-packages/india/page/39',
    'https://traveltriangle.com/family-packages/india/page/40'
]

for url in url_list:
    try:
        page = requests.get(url, allow_redirects=False)
        soup = BeautifulSoup(page.text, 'html.parser')

        package_class = soup.find_all('span', class_="fw9 m0 f16 pfc3")
        package_names.extend([package.text for package in package_class])

        price_class = soup.find_all('h4', class_="sfc3 m0 f24 fw9 priceVal at_newprice")

        if price_class:
            prices.extend([price.text.split()[1].split('/')[0] if len(price.text.split()) > 1 else np.nan for price in price_class])
            prices_before_discount.extend([price.text.split()[2].split('/')[0] if len(price.text.split()) > 2 else np.nan for price in price_class])
        else:
            # If price_class is empty, fill prices and prices_before_discount with NaN
            prices.extend([np.nan])
            prices_before_discount.extend([np.nan])

        duration = soup.find_all('span', class_="iblock sfc6")
        d = [duration.text for duration in duration]
        days.extend([package.split()[0] for package in d])
        nights.extend([package.split()[3] for package in d])

        city_class = soup.find_all('ul', class_="clearfix package-cities-list at_packagecity_list")
        cities.extend([city.text.split('(')[0] if city else np.nan for city in city_class])

        activities_class = soup.find_all('div', class_="col-md-6 pl8 pr0 clearfix package-tag-box")
        activities = [activity.text for activity in activities_class]

        # Loop for Adventure
        for item in activities:
            if 'Adventure' in item:
                adventures.append('Adventure')
            else:
                adventures.append(np.nan)

        # Loop for Nature
        for item in activities:
            if 'Nature' in item:
                natures.append('Nature')
            else:
                natures.append(np.nan)

        # Loop for Hill Stations
        for item in activities:
            if 'Hill station' in item:
                hill_station.append('Hill Stations')
            else:
                hill_station.append(np.nan)

        # Loop for Water Activities
        for item in activities:
            if 'Water Activities' in item:
                water_activities.append('Water Activities')
            else:
                water_activities.append(np.nan)

        # Loop for Religious
        for item in activities:
            if 'Religious' in item:
                religious_list.append('Religious')
            else:
                religious_list.append(np.nan)

        starhotel_class = soup.find_all('div', class_="flex alignCenter")
        stars = [star.text for star in starhotel_class]

        for item in stars:
            # Extract star ratings using regular expression
            ratings = re.findall(r'\d Star', item)

            # If there are star ratings, find the maximum and append to the star_hotels list
            if ratings:
                max_star = max(map(int, re.findall(r'\d', ' '.join(ratings))))
                star_hotels.append(f'{max_star} Star')
            else:
                star_hotels.append(item)

        link_class = soup.find_all('a', class_="block fw7 p8 pl15 pr15 text-center link-pri at_packageviewdetailbtn")

        # Find all matches of the specified pattern
        matches = [re.findall(r'href="([^"]*)"', str(tag))[0] for tag in link_class]
        urls.extend(["https://traveltriangle.com" + match for match in matches])

        reviews, ratings, travellers = [], [], []

        for i in urls:
            try:
                page = requests.get(i, allow_redirects=False)
                soup = BeautifulSoup(page.text, 'html.parser')

                # Find the <p> tag with class "f12 at_package_count"
                p_tag = soup.find('p', class_="f12 at_package_count")

                # Check if the <p> tag is found
                if p_tag:
                    # Find all <span> elements within the <p> tag
                    spans = p_tag.find_all('span')

                    # Check if there are at least three <span> elements
                    if len(spans) >= 3:
                        reviews.append(spans[4].text if spans[4] else np.nan)
                        ratings.append(spans[3].text if spans[3] else np.nan)
                    else:
                        # Fill with NaN
                        reviews.append(np.nan)
                        ratings.append(np.nan)
                else:
                    # Fill with NaN
                    reviews.append(np.nan)
                    ratings.append(np.nan)

                travellers_class = soup.find_all('p', class_="pfc4 f12 text-center m8")

                # Loop through the found elements and append to the travellers list
                for traveller in travellers_class:
                    if traveller:
                        travellers.append(''.join(re.findall(r'\d+', traveller.text)))
                    else:
                        # Append NaN if no information is provided
                        travellers.append(np.nan)

            except requests.RequestException as e:
                print(f"Error fetching URL: {i}, {e}")
                # Fill with NaN
                reviews.append(np.nan)
                ratings.append(np.nan)
                # Append NaN for travellers in case of an error
                travellers.append(np.nan)

        print(f"Scraping completed for URL: {url}")

    except requests.RequestException as e:
        print(f"Error fetching URL: {url}, {e}")


Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/35
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/36
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/38
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/39
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/40


In [81]:
prices.insert(124,np.nan)

In [75]:
star_hotels.pop()

'3 Star'

In [79]:
len(prices_before_discount)

124

In [77]:
d={'Package_Names':package_names,
   'Prices_Before_Discount': prices_before_discount, 
   'Prices_After_Discount':prices,
   'Days':days,
   'Nights':nights, 
   'Cities':cities,
   'Adventures':adventures, 
   'Nature':natures, 
   'Hill_Station':hill_station, 
   'Water_Activities':water_activities,
   'Religious':religious_list, 
   'Star_Hotels':star_hotels,
   'Travellers':travellers,
   'Ratings':ratings,
   'Reviews':reviews}

In [82]:
WebPage31_40=pd.DataFrame(d)

In [83]:
WebPage31_40.to_csv(r"C:\Users\mdimr\OneDrive\Desktop\webscrapping\WebPage31_40.csv")

### Pages 21-25

In [84]:
import pandas as pd
import regex as re
import requests
from bs4 import BeautifulSoup
import numpy as np

package_names, prices, prices_before_discount, days, nights, cities, adventures, natures, hill_station, water_activities, religious_list, star_hotels, ratings, reviews, urls, travellers = [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []

# Define your custom URL list
url_list = [
    'https://traveltriangle.com/family-packages/india/page/21',
    'https://traveltriangle.com/family-packages/india/page/22',
    'https://traveltriangle.com/family-packages/india/page/23',
    'https://traveltriangle.com/family-packages/india/page/24',
    'https://traveltriangle.com/family-packages/india/page/25'
]

for url in url_list:
    try:
        page = requests.get(url, allow_redirects=False)
        soup = BeautifulSoup(page.text, 'html.parser')

        package_class = soup.find_all('span', class_="fw9 m0 f16 pfc3")
        package_names.extend([package.text for package in package_class])

        price_class = soup.find_all('h4', class_="sfc3 m0 f24 fw9 priceVal at_newprice")

        if price_class:
            prices.extend([price.text.split()[1].split('/')[0] if len(price.text.split()) > 1 else np.nan for price in price_class])
            prices_before_discount.extend([price.text.split()[2].split('/')[0] if len(price.text.split()) > 2 else np.nan for price in price_class])
        else:
            # If price_class is empty, fill prices and prices_before_discount with NaN
            prices.extend([np.nan])
            prices_before_discount.extend([np.nan])

        duration = soup.find_all('span', class_="iblock sfc6")
        d = [duration.text for duration in duration]
        days.extend([package.split()[0] for package in d])
        nights.extend([package.split()[3] for package in d])

        city_class = soup.find_all('ul', class_="clearfix package-cities-list at_packagecity_list")
        cities.extend([city.text.split('(')[0] if city else np.nan for city in city_class])

        activities_class = soup.find_all('div', class_="col-md-6 pl8 pr0 clearfix package-tag-box")
        activities = [activity.text for activity in activities_class]

        # Loop for Adventure
        for item in activities:
            if 'Adventure' in item:
                adventures.append('Adventure')
            else:
                adventures.append(np.nan)

        # Loop for Nature
        for item in activities:
            if 'Nature' in item:
                natures.append('Nature')
            else:
                natures.append(np.nan)

        # Loop for Hill Stations
        for item in activities:
            if 'Hill station' in item:
                hill_station.append('Hill Stations')
            else:
                hill_station.append(np.nan)

        # Loop for Water Activities
        for item in activities:
            if 'Water Activities' in item:
                water_activities.append('Water Activities')
            else:
                water_activities.append(np.nan)

        # Loop for Religious
        for item in activities:
            if 'Religious' in item:
                religious_list.append('Religious')
            else:
                religious_list.append(np.nan)

        starhotel_class = soup.find_all('div', class_="flex alignCenter")
        stars = [star.text for star in starhotel_class]

        for item in stars:
            # Extract star ratings using regular expression
            ratings = re.findall(r'\d Star', item)

            # If there are star ratings, find the maximum and append to the star_hotels list
            if ratings:
                max_star = max(map(int, re.findall(r'\d', ' '.join(ratings))))
                star_hotels.append(f'{max_star} Star')
            else:
                star_hotels.append(item)

        link_class = soup.find_all('a', class_="block fw7 p8 pl15 pr15 text-center link-pri at_packageviewdetailbtn")

        # Find all matches of the specified pattern
        matches = [re.findall(r'href="([^"]*)"', str(tag))[0] for tag in link_class]
        urls.extend(["https://traveltriangle.com" + match for match in matches])

        reviews, ratings, travellers = [], [], []

        for i in urls:
            try:
                page = requests.get(i, allow_redirects=False)
                soup = BeautifulSoup(page.text, 'html.parser')

                # Find the <p> tag with class "f12 at_package_count"
                p_tag = soup.find('p', class_="f12 at_package_count")

                # Check if the <p> tag is found
                if p_tag:
                    # Find all <span> elements within the <p> tag
                    spans = p_tag.find_all('span')

                    # Check if there are at least three <span> elements
                    if len(spans) >= 3:
                        reviews.append(spans[4].text if spans[4] else np.nan)
                        ratings.append(spans[3].text if spans[3] else np.nan)
                    else:
                        # Fill with NaN
                        reviews.append(np.nan)
                        ratings.append(np.nan)
                else:
                    # Fill with NaN
                    reviews.append(np.nan)
                    ratings.append(np.nan)

                travellers_class = soup.find_all('p', class_="pfc4 f12 text-center m8")

                # Loop through the found elements and append to the travellers list
                for traveller in travellers_class:
                    if traveller:
                        travellers.append(''.join(re.findall(r'\d+', traveller.text)))
                    else:
                        # Append NaN if no information is provided
                        travellers.append(np.nan)

            except requests.RequestException as e:
                print(f"Error fetching URL: {i}, {e}")
                # Fill with NaN
                reviews.append(np.nan)
                ratings.append(np.nan)
                # Append NaN for travellers in case of an error
                travellers.append(np.nan)

        print(f"Scraping completed for URL: {url}")

    except requests.RequestException as e:
        print(f"Error fetching URL: {url}, {e}")


Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/21
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/22
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/23
Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/24


C:\Users\mdimr\AppData\Local\Temp\ipykernel_16640\3935945425.py:107: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(page.text, 'html.parser')


Scraping completed for URL: https://traveltriangle.com/family-packages/india/page/25


In [85]:
d={'Package_Names':package_names,
   'Prices_Before_Discount': prices_before_discount, 
   'Prices_After_Discount':prices,
   'Days':days,
   'Nights':nights, 
   'Cities':cities,
   'Adventures':adventures, 
   'Nature':natures, 
   'Hill_Station':hill_station, 
   'Water_Activities':water_activities,
   'Religious':religious_list, 
   'Star_Hotels':star_hotels,
   'Travellers':travellers,
   'Ratings':ratings,
   'Reviews':reviews}

In [91]:
len(prices_before_discount)

125

In [92]:
travellers.insert(124,np.nan)

In [93]:
WebPage21_25=pd.DataFrame(d)

In [94]:
WebPage21_25.to_csv(r"C:\Users\mdimr\OneDrive\Desktop\webscrapping\WebPage21_25.csv")